In [ ]:
import requests
import pandas as pd
from tqdm import tqdm  # tqdm 라이브러리 추가
import math

api_key=''

def split_locations(locations, max_locations=10):
    """
    Split locations into smaller groups to avoid exceeding API URL limits.
    
    Args:
        locations (list): List of location names.
        max_locations (int): Maximum number of locations per group.
    
    Returns:
        list: List of grouped locations.
    """
    return [locations[i:i + max_locations] for i in range(0, len(locations), max_locations)]

def calculate_distances(locations, api_key, max_locations=10):
    """
    Calculate distances between all given locations using Google Distance Matrix API.
    
    Args:
        locations (list): List of location names.
        api_key (str): Google Maps API key.
        max_locations (int): Maximum number of locations per group.
        
    Returns:
        pandas.DataFrame: Distance matrix with travel times.
    """
    # Prepare empty matrices
    n = len(locations)
    distance_matrix = [["" for _ in range(n)] for _ in range(n)]
    time_matrix = [["" for _ in range(n)] for _ in range(n)]
    
    # Split locations into manageable groups
    groups = split_locations(locations, max_locations)
    total_iterations = len(groups) ** 2  # total calcuation time

    # Loop through each origin group
    with tqdm(total=total_iterations, desc="Processing Groups") as pbar:
        for i, origins_group in enumerate(groups):
            for j, destinations_group in enumerate(groups):
                # Prepare API endpoint and parameters
                endpoint = "https://maps.googleapis.com/maps/api/distancematrix/json"
                params = {
                    "origins": "|".join(origins_group),
                    "destinations": "|".join(destinations_group),
                    "key": api_key,
                    "units": "metric",
                }
                
                # API call
                try:
                    response = requests.get(endpoint, params=params)
                    response.raise_for_status()  # Raise HTTPError for bad responses
                    data = response.json()
                    
                    # Check for valid response
                    if data.get("status") != "OK":
                        # print(f"Warning: API response error - {data.get('error_message', 'Unknown error')}")
                        continue
                    
                    # Extract distances and durations
                    for oi, row in enumerate(data.get("rows", [])):
                        for di, elem in enumerate(row.get("elements", [])):
                            distance = elem.get("distance", {}).get("text", "")
                            duration = elem.get("duration", {}).get("text", "")
                            origin_index = i * max_locations + oi
                            destination_index = j * max_locations + di
                            if origin_index < n and destination_index < n:
                                distance_matrix[origin_index][destination_index] = distance
                                time_matrix[origin_index][destination_index] = duration
                except Exception as e:
                    # print(f"Error during API call for group ({i}, {j}): {str(e)}")
                    continue
                finally:
                    pbar.update(1)  # Update progress bar for each iteration

    # Create DataFrames
    distance_df = pd.DataFrame(distance_matrix, index=locations, columns=locations)
    time_df = pd.DataFrame(time_matrix, index=locations, columns=locations)
    
    return distance_df, time_df


In [109]:
data = pd.read_excel('Dataset.xlsx')
locations = data['Name'].tolist()
len(locations)

141

In [110]:
# Calculate distances and durations
distance_df, time_df = calculate_distances(locations, api_key, max_locations=5)

print("\nTime Matrix (in hours/minutes):")
display(time_df)

Processing Groups: 100%|██████████| 841/841 [01:40<00:00,  8.36it/s]


Time Matrix (in hours/minutes):


,"60 Bukit Timah Rd, B1-02, Singapore 229900","300 Tiong Bahru Rd, Singapore 168731","26 Bussorah St, Singapore 199444","261 Victoria St, Singapore 189876","10, Telok Blangah Green, 109178",Gardens by the Bay,Wings of Time,Night Safari Singapore,Marina Bay Sands Skypark Observation Deck,Singapore Cable Car,...,"119 Owen Rd, Singapore 218924","1 Vista Exchange Green, #B1-45 The Star Vista, Singapore 138617","6 Holland Cl, #01-36 Block 6, Singapore 271006",Si Chuan Dou Hua Restaurant,"01-01/02/03, Lavender Aperia, 12 Kallang Ave, 339511",Lai Kee Pau & Dim Sum - Clementi,"200 Victoria St, B1 04A, Singapore 188021","36 College Ave E, North Tower, Singapore 138600","10 Bayfront Ave, Singapore 018956","65 Airport Blvd., Level 2 Terminal 3, Singapore 819663"
"60 Bukit Timah Rd, B1-02, Singapore 229900",1 min,14 mins,7 mins,7 mins,19 mins,10 mins,21 mins,28 mins,12 mins,19 mins,...,3 mins,21 mins,18 mins,13 mins,9 mins,19 mins,6 mins,21 mins,11 mins,19 mins
"300 Tiong Bahru Rd, Singapore 168731",14 mins,1 min,15 mins,12 mins,7 mins,9 mins,13 mins,29 mins,13 mins,9 mins,...,13 mins,14 mins,13 mins,9 mins,16 mins,18 mins,13 mins,13 mins,12 mins,22 mins
"26 Bussorah St, Singapore 199444",9 mins,20 mins,1 min,7 mins,19 mins,8 mins,20 mins,30 mins,9 mins,18 mins,...,11 mins,24 mins,21 mins,12 mins,8 mins,22 mins,5 mins,23 mins,9 mins,18 mins
"261 Victoria St, Singapore 189876",5 mins,15 mins,4 mins,1 min,18 mins,7 mins,19 mins,27 mins,10 mins,17 mins,...,7 mins,20 mins,17 mins,11 mins,6 mins,18 mins,4 mins,22 mins,8 mins,17 mins
"10, Telok Blangah Green, 109178",18 mins,8 mins,18 mins,16 mins,1 min,12 mins,14 mins,30 mins,16 mins,10 mins,...,17 mins,14 mins,15 mins,13 mins,20 mins,19 mins,17 mins,14 mins,15 mins,25 mins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lai Kee Pau & Dim Sum - Clementi,17 mins,20 mins,22 mins,22 mins,21 mins,21 mins,26 mins,16 mins,22 mins,23 mins,...,19 mins,13 mins,14 mins,22 mins,19 mins,1 min,22 mins,12 mins,23 mins,26 mins
"200 Victoria St, B1 04A, Singapore 188021",8 mins,16 mins,6 mins,4 mins,20 mins,9 mins,21 mins,31 mins,8 mins,19 mins,...,11 mins,21 mins,21 mins,8 mins,10 mins,22 mins,1 min,21 mins,10 mins,19 mins
"36 College Ave E, North Tower, Singapore 138600",21 mins,13 mins,21 mins,19 mins,14 mins,14 mins,20 mins,25 mins,19 mins,16 mins,...,19 mins,9 mins,10 mins,16 mins,22 mins,11 mins,19 mins,1 min,18 mins,27 mins
"10 Bayfront Ave, Singapore 018956",11 mins,14 mins,8 mins,9 mins,14 mins,4 mins,15 mins,30 mins,4 mins,14 mins,...,13 mins,18 mins,19 mins,10 mins,12 mins,23 mins,7 mins,18 mins,1 min,17 mins


In [111]:
time_df.to_excel('./distance_v4.xlsx')

In [112]:
# hour to min 
def time_to_minutes(value):
    if isinstance(value, str):
        minutes = 0
        try:
            # Days -> min
            if "day" in value:
                days_part = value.split("day")[0].strip()
                minutes += int(days_part) * 24 * 60  # day -> min
                value = " ".join(value.split("day")[1:]).strip()
            
            # Hours -> min
            if "hour" in value:
                hours_part = value.split("hour")[0].strip()
                minutes += int(hours_part) * 60  # hour -> min
                value = " ".join(value.split("hour")[1:]).strip()
            
            # Minutes 
            if "min" in value:
                minutes_part = value.split("min")[0].strip()
                if minutes_part.isdigit():
                    minutes += int(minutes_part)
        except Exception as e:
            print(f"Error processing value '{value}': {e}")
            return ''
        return minutes
    return ''  # N/A -> 0

time_df_minutes = time_df.applymap(time_to_minutes)

/var/folders/pw/_ml9mkrn07v5x0fngqp4qhg00000gn/T/ipykernel_15531/2375099597.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  time_df_minutes = time_df.applymap(time_to_minutes)


In [113]:
time_df_minutes

,"60 Bukit Timah Rd, B1-02, Singapore 229900","300 Tiong Bahru Rd, Singapore 168731","26 Bussorah St, Singapore 199444","261 Victoria St, Singapore 189876","10, Telok Blangah Green, 109178",Gardens by the Bay,Wings of Time,Night Safari Singapore,Marina Bay Sands Skypark Observation Deck,Singapore Cable Car,...,"119 Owen Rd, Singapore 218924","1 Vista Exchange Green, #B1-45 The Star Vista, Singapore 138617","6 Holland Cl, #01-36 Block 6, Singapore 271006",Si Chuan Dou Hua Restaurant,"01-01/02/03, Lavender Aperia, 12 Kallang Ave, 339511",Lai Kee Pau & Dim Sum - Clementi,"200 Victoria St, B1 04A, Singapore 188021","36 College Ave E, North Tower, Singapore 138600","10 Bayfront Ave, Singapore 018956","65 Airport Blvd., Level 2 Terminal 3, Singapore 819663"
"60 Bukit Timah Rd, B1-02, Singapore 229900",1,14,7,7,19,10,21,28,12,19,...,3,21,18,13,9,19,6,21,11,19
"300 Tiong Bahru Rd, Singapore 168731",14,1,15,12,7,9,13,29,13,9,...,13,14,13,9,16,18,13,13,12,22
"26 Bussorah St, Singapore 199444",9,20,1,7,19,8,20,30,9,18,...,11,24,21,12,8,22,5,23,9,18
"261 Victoria St, Singapore 189876",5,15,4,1,18,7,19,27,10,17,...,7,20,17,11,6,18,4,22,8,17
"10, Telok Blangah Green, 109178",18,8,18,16,1,12,14,30,16,10,...,17,14,15,13,20,19,17,14,15,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lai Kee Pau & Dim Sum - Clementi,17,20,22,22,21,21,26,16,22,23,...,19,13,14,22,19,1,22,12,23,26
"200 Victoria St, B1 04A, Singapore 188021",8,16,6,4,20,9,21,31,8,19,...,11,21,21,8,10,22,1,21,10,19
"36 College Ave E, North Tower, Singapore 138600",21,13,21,19,14,14,20,25,19,16,...,19,9,10,16,22,11,19,1,18,27
"10 Bayfront Ave, Singapore 018956",11,14,8,9,14,4,15,30,4,14,...,13,18,19,10,12,23,7,18,1,17


In [114]:
time_df_minutes.to_excel('./distance_in_min_v4.xlsx')